In [ ]:
#pip install gcsfs

In [ ]:
!pip install pyspark
!pip install dask
!pip install "dask[distributed]" --upgrade
!pip3 install torch torchvision torchaudio
!pip install transformers accelerate bitsandbytes
!pip install -U flash-attn --no-build-isolation

In [3]:
import pandas as pd
import re
import json
import pandas as pd
import dask.dataframe as dd


In [ ]:
# import pandas as pd
# import gcsfs

# # Replace with your GCS bucket name and folder path
# bucket_name = "stack_overflow_data_1709"
# folder_path = "stack_overflow_data/"  # Folder containing the 900 CSVs

# # Initialize GCS filesystem
# fs = gcsfs.GCSFileSystem()

# # List all CSV files in the folder
# csv_files = fs.ls(f"{bucket_name}/{folder_path}")

# # Filter out files that are not CSVs (if needed)
# csv_files = [file for file in csv_files if file.endswith(".csv")]

# # Initialize an empty list to store DataFrames
# dataframes = []

# # Load each CSV into a DataFrame and append to the list
# for file in csv_files:
#     with fs.open(file) as f:
#         df = pd.read_csv(f)
#         dataframes.append(df)

# # Concatenate all DataFrames into one
# final_dataframe = pd.concat(dataframes, ignore_index=True)

# # Display the resulting DataFrame
# print(final_dataframe)

# # Save the combined DataFrame to a new CSV (optional)
# final_dataframe.to_csv("stack_overflow.csv", index=False)


In [1]:
from dask.distributed import Client

# Start a Dask cluster (default: uses all available CPU cores)
client = Client(n_workers=3)
client
# Check client and resources



Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:8787/status,
Dashboard: http://127.0.0.1:8787/status,Workers: 3
Total threads: 33,Total memory: 125.80 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:33005,Workers: 3
Dashboard: http://127.0.0.1:8787/status,Total threads: 33
Started: Just now,Total memory: 125.80 GiB
Comm: tcp://127.0.0.1:42017,Total threads: 11
Dashboard: http://127.0.0.1:35883/status,Memory: 41.93 GiB
Nanny: tcp://127.0.0.1:34667,


In [ ]:
def preprocess_text(row):
    """
    Preprocess a single row of the dataset: clean text, add special tokens, and wrap code snippets in <CODE> tags.
    """
    question = row["question"]
    answer = row["answer"]
    
    # Define regex patterns for various code scenarios
    code_block_pattern = r"```(.*?)```"  # Match Markdown-style code blocks
    inline_code_pattern = r"<code>(.*?)</code>"  # Match inline HTML <code> tags
    pre_code_pattern = r"<pre><code>(.*?)</code></pre>"  # Match <pre><code> blocks

    # Remove general HTML tags, except for <code> and <pre>
    question_cleaned = re.sub(r"<(?!/?code|/?pre)[^>]+>", "", question)
    answer_cleaned = re.sub(r"<(?!/?code|/?pre)[^>]+>", "", answer)

    # Wrap code blocks with <CODE> markers (Markdown-style)
    question_cleaned = re.sub(code_block_pattern, r"<CODE>\1</CODE>", question_cleaned, flags=re.DOTALL)
    answer_cleaned = re.sub(code_block_pattern, r"<CODE>\1</CODE>", answer_cleaned, flags=re.DOTALL)

    # Wrap inline HTML <code> tags with <CODE> markers
    question_cleaned = re.sub(inline_code_pattern, r"<CODE>\1</CODE>", question_cleaned, flags=re.DOTALL)
    answer_cleaned = re.sub(inline_code_pattern, r"<CODE>\1</CODE>", answer_cleaned, flags=re.DOTALL)

    # Wrap <pre><code> blocks with <CODE> markers
    question_cleaned = re.sub(pre_code_pattern, r"<CODE>\1</CODE>", question_cleaned, flags=re.DOTALL)
    answer_cleaned = re.sub(pre_code_pattern, r"<CODE>\1</CODE>", answer_cleaned, flags=re.DOTALL)

    # Detect standalone lines that look like code based on indentation or special characters
    # Generalized pattern for potential code-like lines
    

    # Remove excessive whitespace and normalize newlines
    question_cleaned = re.sub(r"\n+", "\n", question_cleaned).strip()
    answer_cleaned = re.sub(r"\n+", "\n", answer_cleaned).strip()

    # Add <QUESTION> and <ANSWER> tags
    prompt = f"<QUESTION>{question_cleaned}<QUESTION>"
    completion = f"<ANSWER>{answer_cleaned}<ANSWER>"

    # Return the structured prompt and completion
    return {"prompt": prompt, "completion": completion}



In [ ]:
def preprocess_partition(partition):
    """
    Preprocess an entire Dask DataFrame partition.
    """
    # Apply preprocessing to each row and return a new DataFrame
    processed_rows = [preprocess_text(row) for _, row in partition.iterrows()]
    return pd.DataFrame(processed_rows)  # Convert list of dicts to DataFrame

# Function to preprocess the entire dataset using Dask
def preprocess_dataset_dask(dataset_path, output_path):
    """
    Preprocess a large dataset using Dask for parallelism and save it in JSONL format.
    """
    meta = pd.DataFrame({"prompt": pd.Series(dtype="str"), "completion": pd.Series(dtype="str")})
    # Load dataset using Dask (assuming CSV format)
    ddf = dd.read_csv(
        dataset_path,
        header=None,
        names=["question_id", "question", "answer", "tags"],
        dtype={"question_id": "str", "question": "str", "answer": "str", "tags": "str"}
    )
    print(ddf.shape[0].compute())
    # Apply the preprocessing function to each partition
    processed_data = ddf.map_partitions(preprocess_partition,meta=meta)

    # Compute and save as JSONL
    processed_data = processed_data.compute()
    with open(output_path, "w") as f:
        for _, row in processed_data.iterrows():
            json.dump(row.to_dict(), f)
            f.write("\n")

    print(f"Preprocessed data saved to {output_path}")

In [ ]:
output_path = "finetuning_dataset.jsonl"
preprocess_dataset_dask(dataset_path, output_path)


In [ ]:
df=pd.read_json("finetuning_dataset.jsonl",lines=True)

In [ ]:
df.iloc[0]["prompt"]

In [14]:
import re
def clean_html_with_code(text):
    code_block_pattern = r"```(.*?)```"  # Match Markdown-style code blocks
    inline_code_pattern = r"<code>(.*?)</code>"  # Match inline HTML <code> tags
    pre_code_pattern = r"<pre><code>(.*?)</code></pre>"  # Match <pre><code> blocks
    # Remove general HTML tags, except for <code> and <pre>
    text_cleaned = re.sub(r"<(?!/?code|/?pre)[^>]+>", "", text)

    # Wrap code blocks with <CODE> markers (Markdown-style)
    text_cleaned = re.sub(code_block_pattern, r"<CODE>\1</CODE>", text_cleaned, flags=re.DOTALL)

    # Wrap inline HTML <code> tags with <CODE> markers
    text_cleaned = re.sub(inline_code_pattern, r"<CODE>\1</CODE>", text_cleaned, flags=re.DOTALL)

    # Wrap <pre><code> blocks with <CODE> markers
    text_cleaned = re.sub(pre_code_pattern, r"<CODE>\1</CODE>", text_cleaned, flags=re.DOTALL)

    # Detect standalone lines that look like code based on indentation or special characters
    # Generalized pattern for potential code-like lines
    

    # Remove excessive whitespace and normalize newlines
    text_cleaned = re.sub(r"\n+", "\n", text_cleaned).strip()
    return text_cleaned
# Function to preprocess a single row
def preprocess_row(row):
    """
    Preprocesses a single row containing question and answer fields.
    Cleans the text, removes unwanted HTML tags, and preserves code blocks.
    """
    question_text = clean_html_with_code(str(row["question"]))
    answer_text = clean_html_with_code(str(row["answer"]))

    # Return formatted input for training
    return {
        "prompt": f"<QUESTION>{question_text}<QUESTION>",
        "completion": f"<ANSWER>{answer_text}<ANSWER>"
    }
dataset_path = "gs://stack_overflow_data_1709/stack_overflow_data/*.csv"  # Replace with your dataset file path
meta = pd.DataFrame({"prompt": pd.Series(dtype="str"), "completion": pd.Series(dtype="str")})
ddf = dd.read_csv(
        dataset_path,
        header=None,
        names=["question_id", "question", "answer", "tags"],
        dtype={"question_id": "str", "question": "str", "answer": "str", "tags": "str"}
    )

# Apply preprocessing across the Dask DataFrame
processed_ddf = ddf.map_partitions(lambda partition: partition.apply(preprocess_row, axis=1))
processed_ddf=processed_ddf.compute()

In [22]:
preprocessed_list=processed_ddf.to_list()
fine_tuning_df = pd.DataFrame(preprocessed_list) 

In [28]:
# Save to a file
with open('fine_tuning_dataset.jsonl', 'w') as f:
    f.write(fine_tuning_df.to_json(orient='records', lines=True))

In [29]:
df=pd.read_json("fine_tuning_dataset.jsonl",lines=True)

In [ ]:
df.iloc[0]["prompt"]

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
import torch

model_name = "codellama/CodeLlama-13b-Instruct-hf"

# Configure 4-bit quantization
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=True,
)

# Load the tokenizer and model
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map="cuda",
)


In [ ]:
prompt = """I have a dockerfile that is suppsed to start the tomcat. I want to create and not run the image from that dockerfile so that in docker images I can see the image. docker run -it logs me in to container but I dont want to create container and log in to it.

I just want to create a image from dockerfile (without creating a container of it).

By which command can I achieve this ?"""

input_ids = tokenizer(prompt, return_tensors="pt").input_ids.to("cuda")

gen_tokens = model.generate(
    input_ids,
    do_sample=True,
    temperature=0.9, top_p=0.95, max_new_tokens=512
)
gen_text = tokenizer.batch_decode(gen_tokens)[0]

In [ ]:
gen_text

In [2]:
import pandas as pd
df=pd.read_json("fine_tuning_dataset.jsonl",lines=True,nrows=100)

In [4]:
df.iloc[0]["prompt"]

'<QUESTION>Struggling to understand how to achieve this, even though it\'s extremely commonplace across the web. \nI have two modals, one is a "sign up" modal, the other is a "log in" modal. I need to be able to perform both actions via a user\'s Google account. I am successfully creating and logging in users via the Google API.\nThe trouble comes with the fact that Google\'s drop-in button automatically signs the user in. \nOn the page I have:\n<pre><CODE>&lt;div class="g-signin2" data-onsuccess="googleSignUp"&gt;&lt;/div&gt;\n</CODE></pre>\nAnd later:\n<pre><CODE>&lt;div class="g-signin2" data-onsuccess="googleLogIn"&gt;&lt;/div&gt;\n</CODE></pre>\nObviously these two buttons have different <CODE>onsuccess</CODE> functions, but both are being called when the user is logged in. I have somewhat alleviated the problem by only actually getting the Google script on a button click:\n<pre><CODE>$(\'a#google-login\').click(function() {\n    $.getScript(\'https://apis.google.com/js/platform.j